In [1]:
import keras
from pyspark import SparkContext, SparkConf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
conf = SparkConf().setAppName("app-name-of-your-choice").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [3]:
pythonList = [2.3, 3.4, 4.3, 2.4, 2.4, 4.0]

In [4]:
parPthonData = sc.parallelize(pythonList, 2)

In [5]:
tempData = [59, 57.2, 53.6, 55.4, 51.0, 53.6, 55.4]

In [6]:
parTempData = sc.parallelize(tempData, 2)

In [7]:
parTempData.collect()

[59, 57.2, 53.6, 55.4, 51.0, 53.6, 55.4]

In [14]:
print(type(parTempData))
print(parTempData.collect()[0])

<class 'pyspark.rdd.RDD'>
59


In [8]:
def fahrenheitToCentigrade(temperature):
    centigrade = (temperature - 32) * 5 /9
    return centigrade(59)

### Text data

In [18]:
rdd = sc.textFile('for_pyspark.txt')

In [19]:
words = rdd.flatMap(lambda x: x.split(" "))

In [20]:
result = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

In [21]:
words.collect()

['This', 'is', 'a', 'textbook', 'This', 'textbook', 'is', 'about', 'birds']

In [23]:
result.collect()

[('is', 2), ('birds', 1), ('This', 2), ('a', 1), ('textbook', 2), ('about', 1)]

In [2]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession \
    .builder \
    .appName("Python Spark regression example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [5]:
df = spark.read.csv("Advertising.csv", header=True, inferSchema=True)

In [6]:
df.show(5)

+---+-----+-----+---------+-----+
|_c0|   TV|radio|newspaper|sales|
+---+-----+-----+---------+-----+
|  1|230.1| 37.8|     69.2| 22.1|
|  2| 44.5| 39.3|     45.1| 10.4|
|  3| 17.2| 45.9|     69.3|  9.3|
|  4|151.5| 41.3|     58.5| 18.5|
|  5|180.8| 10.8|     58.4| 12.9|
+---+-----+-----+---------+-----+
only showing top 5 rows



In [7]:
df.count()

200

In [9]:
df.t

+-------+------------------+-----------------+------------------+------------------+------------------+
|summary|               _c0|               TV|             radio|         newspaper|             sales|
+-------+------------------+-----------------+------------------+------------------+------------------+
|  count|               200|              200|               200|               200|               200|
|   mean|             100.5|         147.0425|23.264000000000024|30.553999999999995|14.022500000000003|
| stddev|57.879184513951124|85.85423631490805|14.846809176168728| 21.77862083852283| 5.217456565710477|
|    min|                 1|              0.7|               0.0|               0.3|               1.6|
|    max|               200|            296.4|              49.6|             114.0|              27.0|
+-------+------------------+-----------------+------------------+------------------+------------------+



In [10]:
from pyspark.mllib.regression import LabeledPoint

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [11]:
regressionRDD = df.rdd.map(list)

In [14]:
point = regressionRDD.map(lambda data : LabeledPoint(data[3, data[0:3]]))

In [15]:
point.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 9, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/anaconda3/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/anaconda3/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/anaconda3/lib/python3.6/site-packages/pyspark/rdd.py", line 1354, in takeUpToNumLeft
    yield next(iterator)
  File "/anaconda3/lib/py